In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.0.0 cuda: True


In [7]:
## Load dataset
data_dir = '/home/hao/Research/apg_data'
data_path = data_dir + "/gmm/gmm_10size"
Data = []
for i in range(5):
    data = torch.from_numpy(np.load(data_path + '/obs_%d.npy' % ((i+1)*20))).float()
    Data.append(data)
K = 3 ## number of clusters
D = 2
NUM_HIDDEN_LOCAL = 32
NUM_HIDDEN_GLOBAL = 32
CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda:1')

PATH = 'lstm-60pts-10samples'
B=20
SAMPLE_SIZE = 1000
NUM_LAYERS = 2 

Data_test = []
Ns = [120, 160, 200, 400, 600] 
for i in range(5):
    data = torch.from_numpy(np.load('/home/hao/Research/ncmm/rings_%d/' % (Ns[i]) + 'ob_%d.npy' % (Ns[i]))).float()
    Data_test.append(data)

In [8]:
from model_operations_lstm import Init_models_test
models = Init_models_test(K, D, B, SAMPLE_SIZE, NUM_HIDDEN_GLOBAL, NUM_HIDDEN_LOCAL, NUM_LAYERS, CUDA, DEVICE, lr=1e-4, RESTORE=True, PATH=PATH)

In [9]:
from vae import VAE_test
S = 1000
B = 20
MCMC_STEPS = 15
Test_Params = (S, B, DEVICE)

def Test_ALL(objective, data_g, mcmc_steps, Test_Params):
    Metrics = {'ll' : [], 'ess' : []}
    (S, B, DEVICE) = Test_Params
    EPS = torch.FloatTensor([1e-15]).log() ## EPS for KL between categorial distributions
    EPS = EPS.cuda().to(DEVICE) ## EPS for KL between categorial distributions
    NUM_DATASETS = data_g.shape[0]
    NUM_BATCHES = int((NUM_DATASETS / B))
    for step in range(NUM_BATCHES):
        ob = data_g[step*B : (step+1)*B]
        ob = shuffler(ob).repeat(S, 1, 1, 1)
        ob = ob.cuda().to(DEVICE)
        metrics = objective(models, ob, mcmc_steps)
        Metrics['ess'].append(metrics['ess'] / S)
        Metrics['ll'].append(metrics['ll'])
    return Metrics

In [5]:
for g, data_g in enumerate(Data):
    time_start = time.time()
    Metrics = Test_ALL(VAE_test, data_g, 0, Test_Params)
    for key in Metrics.keys():
        np.save('lstm_%d-%s' % (((g+1)*20), key), torch.cat(Metrics[key], 0))
    time_end = time.time()    
    print('group=%d (%ds)' % (g, time_end - time_start))

In [ ]:
torch.cat(Metrics['ll'], 0).mean(0)

In [ ]:
LLs = []
for i in range(5):
    log_j = np.load('lstm_data_%i-ll.npy' % i).mean(0)
    LLs.append(log_j)

In [ ]:
np.array(LLs).mean()

In [ ]:
from testing import Eval
from viz_mc import Viz_MC
from vae import VAE_test

BATCH_SIZE = 5
Viz_INTERVAL = 4
FS = 5
TITLE_FS = 20
OB_MS = 5.0
MU_MARKER = 'X'
MU_MS = 100
OPACITY = 0.3
BOUND = 12
COLORS = ['#AA3377', '#EE7733', '#0077BB', '#009988']
DATA_PTR = 73

In [ ]:
visualizer = Viz_MC(K, Viz_INTERVAL, FS, TITLE_FS, OB_MS, OPACITY, BOUND, COLORS)
evaluator = Eval(models, K, D, BATCH_SIZE, CUDA, DEVICE)

Metrics_sample = evaluator.Test_uniform(VAE_test, Data, DATA_PTR, mcmc_steps=None, sample_size=1)
visualizer.Plot_chains(Metrics_sample['data'], Metrics_sample['samples'], filename='gmm-samples-%d' % DATA_PTR)

In [ ]:
# L = 1000
# Metrics_converge = evaluator.Test_uniform(APG_test, Data, DATA_PTR, mcmc_steps=30, sample_size=L)
# visualizer.Plot_metrics(Metrics_converge['log_joint'], Metrics_converge['ess'], sample_size=L, filename='gmm-metrics-LL-ESS-%d' % DATA_PTR)

In [ ]:
Metrics_sample.keys()


In [ ]:
aa = evaluator.Sample_data_uniform(Data, DATA_PTR)

In [ ]:
aa[3]